# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,adamstown,-25.0660,-130.1015,23.01,83,58,5.63,PN,1733238451
1,1,moranbah,-22.0016,148.0466,23.84,76,100,5.96,AU,1733238452
2,2,margaret river,-33.9500,115.0667,17.35,77,94,6.19,AU,1733238453
3,3,qalat,32.1058,66.9083,8.39,42,1,2.69,AF,1733238455
4,4,mbandaka,0.0487,18.2603,30.15,57,44,1.83,CD,1733238456


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color = "City",
    alpha = 0.5
)

# Display the map
map_plot


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df.loc[(city_data_df["Max Temp"] < 30) & (city_data_df["Max Temp"] > 21)] # Narrow down to temperature between 18 to 26C
ideal_weather_df = ideal_weather_df.loc[(ideal_weather_df["Humidity"] < 80) & (ideal_weather_df["Humidity"] > 30)] # Narrow down to humidity between 30 to 70%
ideal_weather_df = ideal_weather_df.loc[ideal_weather_df["Wind Speed"] < 10] # Narrow down to speed less than 10m/s.
ideal_weather_df = ideal_weather_df.loc[ideal_weather_df["Cloudiness"] == 0] # Narrow down to cloudiness = 0%

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,5,holualoa,19.6228,-155.9522,22.15,37,0,2.06,US,1733238363
28,28,carnarvon,-24.8667,113.6333,21.81,73,0,9.48,AU,1733238484
46,46,cabo san lucas,22.8909,-109.9124,21.17,52,0,2.57,MX,1733238491
63,63,dom pedrito,-30.9828,-54.6731,21.72,41,0,4.53,BR,1733238530
175,175,caluula,11.9661,50.7569,26.14,62,0,5.25,SO,1733238670
181,181,cam ranh,11.9214,109.1591,26.98,69,0,6.69,VN,1733238678
234,234,coahuayana de hidalgo,18.7000,-103.6583,26.28,61,0,0.56,MX,1733238744
400,400,pimentel,-6.8367,-79.9342,22.16,73,0,4.63,PE,1733238947
483,483,tsiombe,-25.3000,45.4833,29.66,52,0,3.58,MG,1733239084


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:

# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
5,5,holualoa,19.6228,-155.9522,22.15,37,0,2.06,US,1733238363,
28,28,carnarvon,-24.8667,113.6333,21.81,73,0,9.48,AU,1733238484,
46,46,cabo san lucas,22.8909,-109.9124,21.17,52,0,2.57,MX,1733238491,
63,63,dom pedrito,-30.9828,-54.6731,21.72,41,0,4.53,BR,1733238530,
175,175,caluula,11.9661,50.7569,26.14,62,0,5.25,SO,1733238670,
181,181,cam ranh,11.9214,109.1591,26.98,69,0,6.69,VN,1733238678,
234,234,coahuayana de hidalgo,18.7000,-103.6583,26.28,61,0,0.56,MX,1733238744,
400,400,pimentel,-6.8367,-79.9342,22.16,73,0,4.63,PE,1733238947,
483,483,tsiombe,-25.3000,45.4833,29.66,52,0,3.58,MG,1733239084,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 100
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address =  requests.get(base_url, params = params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"


        # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
holualoa - nearest hotel: No hotel found
carnarvon - nearest hotel: No hotel found
cabo san lucas - nearest hotel: No hotel found
dom pedrito - nearest hotel: No hotel found
caluula - nearest hotel: No hotel found
cam ranh - nearest hotel: No hotel found
coahuayana de hidalgo - nearest hotel: No hotel found
pimentel - nearest hotel: No hotel found
tsiombe - nearest hotel: No hotel found


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
5,5,holualoa,19.6228,-155.9522,22.15,37,0,2.06,US,1733238363,No hotel found
28,28,carnarvon,-24.8667,113.6333,21.81,73,0,9.48,AU,1733238484,No hotel found
46,46,cabo san lucas,22.8909,-109.9124,21.17,52,0,2.57,MX,1733238491,No hotel found
63,63,dom pedrito,-30.9828,-54.6731,21.72,41,0,4.53,BR,1733238530,No hotel found
175,175,caluula,11.9661,50.7569,26.14,62,0,5.25,SO,1733238670,No hotel found
181,181,cam ranh,11.9214,109.1591,26.98,69,0,6.69,VN,1733238678,No hotel found
234,234,coahuayana de hidalgo,18.7000,-103.6583,26.28,61,0,0.56,MX,1733238744,No hotel found
400,400,pimentel,-6.8367,-79.9342,22.16,73,0,4.63,PE,1733238947,No hotel found
483,483,tsiombe,-25.3000,45.4833,29.66,52,0,3.58,MG,1733239084,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color = "Hotel Name",
    alpha = 0.5,
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Hotel Name,Humidity,Country)